In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType

In [0]:
movie_genre_schema = StructType([
    StructField("movieId", IntegerType()),
    StructField("genreId", IntegerType())])
movie_genre_df = spark.read.json(f"{bronze_folder_path}/{v_file_date}/movie_genre.json", schema=movie_genre_schema)

In [0]:
from pyspark.sql.functions import current_timestamp, lit

movie_genre_final_df = movie_genre_df.withColumnsRenamed({"movieId":"movie_id", "genreId":"genre_id"})\
    .withColumns({"ingestion_date" : current_timestamp(), "enviroment" : lit(v_environment), "file_date" : lit(v_file_date)})

In [0]:
#overwrite_partition(movie_genre_final_df, "movie_silver", "movie_genre")

In [0]:
merge_condicion = "tgt.movie_id = src.movie_id AND tgt.genre_id = src.genre_id AND tgt.file_date = src.file_date"
merge_delta_lake(movie_genre_final_df, "movie_silver", "movie_genre", silver_folder_path, merge_condicion, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.movie_genre
GROUP BY file_date;

In [0]:
display(spark.read.format("delta").load("/mnt/moviehistoryrodrigopenna/silver/movie_genre"))